In [1]:
!pip install pandas
!pip install nltk
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from nltk.tokenize import RegexpTokenizer
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [ ]:
from collections import Counter
import operator

splitter = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:



file_name="data2.csv"

# Read File
df=pd.read_csv(file_name)
# df





df=pd.read_csv(file_name)
documentname_list=list(df['label_dis'])
print(documentname_list)
df=df.iloc[:,1:]
columnsName=list(df.columns)

# print(columnsName)

documentname_list=list(documentname_list)

# df





Num_of_diseases=len(df) 
Num_of_symptoms=len(columnsName)
print(Num_of_diseases,Num_of_symptoms)

In [ ]:

inverted_document_frequency={}

# inverted_document_frequency for diseases in the dataset
for col in columnsName:
  temp=np.count_nonzero(df[col])
#   print(df[col])
  inverted_document_frequency[col]=np.log(Num_of_diseases/temp)


In [ ]:

# term Frequency for all diseases in dataset
term_frequency={}
# df.loc[0,'abdominal cramp']

for i in range(Num_of_diseases):
  for col in columnsName:
    key=(documentname_list[i],col)
    term_frequency[key]=df.loc[i,col]
# term_frequency   

In [ ]:

# Tf-inverted_document_frequency Score for [Document(Disease Name),Symptom name] as-key  and  "weight of the disease word" as-value
term_frequency_inverted_document_frequency={}
for i in range(Num_of_diseases):
  for col in columnsName:
    key=(documentname_list[i],col)
    term_frequency_inverted_document_frequency[key]=float(inverted_document_frequency[col])*float(term_frequency[key])
#     print(str(key)+"=>"+str(term_frequency_inverted_document_frequency[key]))
term_frequency_inverted_document_frequency

In [ ]:
# term_frequency-inverted_document_frequency score 2-d array for "each symptom with disease name"
D_2d = np.zeros((Num_of_diseases, Num_of_symptoms),dtype='float32')
for i in term_frequency_inverted_document_frequency:
    sym = columnsName.index(i[1])
    dis=documentname_list.index(i[0])
    D_2d[dis][sym] = term_frequency_inverted_document_frequency[i]
D_2d

In [ ]:
df = pd.read_csv(file_name)
Y = df.iloc[:, 0:1]
X = df.iloc[:, 1:]


In [ ]:
symptoms_dataset = list(X.columns)
# list of all Symptoms
symptoms_dataset

In [ ]:

diseases = list(set(Y['label_dis']))
# List of all Diseases

diseases.sort()
diseases
# len(diseases)

In [ ]:
user_symptoms = str(input("\nPlease enter symptoms separated by comma(,):\n")).lower().split(',')
user_symptoms

In [ ]:
processed_user_symptoms=[]
for sym in user_symptoms:
    sym=sym.strip()
    sym=sym.replace('-',' ')
    sym=sym.replace("'",'')
    sym = ' '.join([lemmatizer.lemmatize(word) for word in splitter.tokenize(sym)])
    processed_user_symptoms.append(sym)
processed_user_symptoms

In [ ]:
user_symptoms=processed_user_symptoms

In [ ]:
symptoms_matched = set()
for idx, data in enumerate(symptoms_dataset):
    data_split=data.split()
    for user_sym in user_symptoms:
        count=0
        for symp in data_split:
            if symp in user_sym.split():
                count+=1
        if count/len(data_split)>0.5:
            symptoms_matched.add(data)
symptoms_matched = list(symptoms_matched)
symptoms_matched

In [ ]:
print(symptoms_matched)
# retreive the whole set of diseases in (set_diseases) containing the symptoms in the "symptoms_matched"
set_diseases = set()
finalSymptoms = [] 
MatchedDiseases = []
for idx in symptoms_matched:
    
    finalSymptoms.append(idx)
    set_diseases.update(set(df[df[idx]==1]['label_dis']))

print("diseaseList")
set_diseases

In [ ]:
# from diseaseList List the whole Symptoms in it
for dis in set_diseases:
    row = df.loc[df['label_dis'] == dis].values.tolist()
    row[0].pop(0)
    for idx,val in enumerate(row[0]):
        if val!=0 and symptoms_dataset[idx] not in finalSymptoms:
            MatchedDiseases.append(symptoms_dataset[idx])
print("whole set of symptoms")
MatchedDiseases

In [ ]:
# from whole list of diseases count the occurrences and store it in a tuple
dict_symp = dict(Counter(MatchedDiseases))
dict_symp
dict_symp_tup = sorted(dict_symp.items(), key=operator.itemgetter(1),reverse=True)
print(dict_symp_tup)
print("-----------------------------------------------")
alldis=[]
# disease List
for i in dict_symp_tup:
    alldis.append(i[0])
print(alldis)

In [ ]:
symptoms_matched=[]
counter=0
# print(finalSymptoms)


# append top ten most occurring symptoms into finalSymptomstomsList
    
for i in dict_symp_tup:
  if counter<10:
    symptoms_matched.append(i[0])
    counter+=1
  else :
    break
print(symptoms_matched)
for idx in symptoms_matched:
    if idx not in finalSymptoms:
            finalSymptoms.append(idx)
finalSymptoms

In [ ]:
# refine the input of symptoms by adding the more Symptoms to it
input2=input("enter some more symptoms from printed above Symptoms:")
arr3=input2.split(",")
for i in arr3:
    finalSymptoms.append(i)
finalSymptoms

In [ ]:
count = 10
# print the top 10 documents based term_frequency_inverted_document_frequency score in between the "query" word and the term_frequency_inverted_document_frequency of the disease word
def term_frequency_inverted_document_frequency_score(count, query):
    query_weights = {}
    for key in term_frequency_inverted_document_frequency:
        
        if key[1] in query:
            try:
                query_weights[key[0]] += term_frequency_inverted_document_frequency[key]
            except:
                query_weights[key[0]] = term_frequency_inverted_document_frequency[key]
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
  
    l = []
    for i in query_weights[:count]:
        l.append(i)
    return l
tf_idf_scored_docs=term_frequency_inverted_document_frequency_score(count,finalSymptoms)
print(tf_idf_scored_docs)


In [ ]:
print(" Diseases Based on Tf-Idf Scoring")
i = 0
tf_idf_scored_docs_index_mapping = {}
for key, val in tf_idf_scored_docs:
#   print(f"{i}. Disease : {key} \t Score : {round(score, 2)}")
  print( f"{key} =======> {round(val, 2)}")
  tf_idf_scored_docs_index_mapping[i] = key
  


In [ ]:
def cosine_dot(a, b):
    if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return 0
    else:
        temp = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
        return temp
    
def gen_vector(tokens):
    Q = np.zeros(Num_of_symptoms)
    counter = Counter(tokens)
    query_weights = {}
    for token in np.unique(tokens):
        term_frequency = counter[token]
        try:
          inverted_document_frequency_temp=inverted_document_frequency[token]
        except:
          pass
        try:
            ind = columnsName.index(token)
            Q[ind] = term_frequency*inverted_document_frequency_temp
        except:
            pass
    return Q  

def cosine_similarity(count, query):
    d_cosines = []
    query_vector = gen_vector(query)
    for d in D_2d:
        d_cosines.append(cosine_dot(query_vector, d))
    out = np.array(d_cosines).argsort()[-count:][::-1]
  
    final_display_disease={}
    for lt in set(out):
      final_display_disease[lt] = float(d_cosines[lt])
    return final_display_disease

cosine_similiarity_docs1=cosine_similarity(count,finalSymptoms)

In [ ]:

print(" Disease based on Cosine Similarity ")

print(cosine_similiarity_docs1)

cosine_similiarity_docs1_sorted = dict(sorted(cosine_similiarity_docs1.items(), key=lambda kv: kv[1], reverse=True))

print(cosine_similiarity_docs1_sorted)
print()
j = 0
cosine_similiarity_docs1_index_mapping = {}
for key in cosine_similiarity_docs1_sorted:
  print(f" {diseases[key]} =====> {round(cosine_similiarity_docs1_sorted[key], 2)}")
  cosine_similiarity_docs1_index_mapping[j] = diseases[key]
  

In [ ]:
#pseudo relevance feedback

diseases
dd_keys=list(cosine_similiarity_docs1_sorted.keys())
dd=[]
print(dd_keys)
    
j=0    
for i in dd_keys:
    if j<5:
        dd.append(i)
        j+=1
print(dd)

symp=[]



for i in dd:
    for col in columnsName:
        if(df.loc[i,col]!=0):
         symp.append(col)
symp=list(set(symp))
symp

In [ ]:

cosine_similiarity_docs3=cosine_similarity(count,symp)


# In[108]:


print("Top most 10 Refined  Diseases  on Cosine Similarity ")
print()
print(cosine_similiarity_docs3)

cosine_similiarity_docs3_sorted = dict(sorted(cosine_similiarity_docs3.items(), key=lambda kv: kv[1], reverse=True))

print(cosine_similiarity_docs3_sorted)
print()


cosine_similiarity_docs3_index_mapping = {}
for key in cosine_similiarity_docs3_sorted:
  print(f" {diseases[key]} =========> {round(cosine_similiarity_docs3_sorted[key], 2)}")
  cosine_similiarity_docs3_index_mapping[j] = diseases[key]
  

In [ ]:

#Accuracy
# diseases
dd_keys=list(cosine_similiarity_docs1_sorted.keys())
dd2=[]
print(dd_keys)
for i in dd_keys:
   dd2.append(diseases[i])
# print(dd2)



dd_keys=list(cosine_similiarity_docs3_sorted.keys())
dd3=[]
print(dd_keys)
for i in dd_keys:
   dd3.append(diseases[i])
# print(dd3)


# In[110]:



#Accuracy

def common(a,b): 
    c = [value for value in a if value in b] 
    return c

d=common(dd2,dd3)
print("Accuracy Before and after Pseudo Relevance Feedback is :",(len(d)/10) *100)


In [ ]:
# plotting in graph
diseases_lst = dd3
 
data = list(cosine_similiarity_docs3_sorted.values())
 

fig = plt.figure(figsize =(10, 7))
plt.pie(data, labels = diseases_lst)

plt.show()

x = dd3
y = list(cosine_similiarity_docs3_sorted.values())

plt.barh(x, y)
plt.show()